In [1]:
from IPython.display import clear_output
!pip install -e git+https://github.com/projectmesa/mesa#egg=mesa
import mesa
!pip install ipynb
import ipynb
clear_output(wait=True)
print("Everything A-Okay!")

Everything A-Okay!


In [2]:
import numpy as np
import math

In [13]:
from mesa import Model
from mesa.space import ContinuousSpace

class TestModel(Model):
    def __init__(self, width, height):
        self.height = width
        self.width = height
        
        # Create continuous space
        self.space = ContinuousSpace(self.width, self.height, torus=False, x_min=0, y_min=0)
        
        self.n_agents = 0
        self.agents = []

    
    def new_agent(self, agent_type, pos, angle):
        '''
        Method that enables us to add agents of a given type.
        '''
        self.n_agents += 1
        
        # Create a new agent of the given type
        new_agent = agent_type(self.n_agents, self, pos, angle)
        
        # Place the agent on the grid
        self.space.place_agent(new_agent, pos)
        
        # And add the agent to the model so we can track it
        self.agents.append(new_agent)
        
    def remove_agent(self, agent):
        '''
        Method that enables us to remove passed agents.
        '''
        self.n_agents -= 1
        
        # Remove agent from grid
        self.space.remove_agent(agent)
        
        # Remove agent from model
        self.agents.remove(agent)
        
    def step(self):
        '''
        Method that steps every agent. 
        
        Prevents applying step on new agents by creating a local list.
        '''
        for agent in list(self.agents):
            agent.step()

In [82]:
from mesa import Agent
import random

class Monkey(Agent):
    def __init__(self, unique_id, model, pos, angle):
        super().__init__(unique_id, model)

        self.pos = pos
        self.pre_pos = pos
#         self.vis_range = 3
#         self.vision_angle = 3.14
        self.direction = angle

    def move(self, new_pos):
        '''
        This method should get the neighbouring cells (Moore's neighbourhood), select one, and move the agent to this cell.
        '''
        
        # Move agent to the new position
        self.model.space.move_agent(self, new_pos)
        
#     def peds_in_angle(self,  vision_angle, vis_range, neighbours):
        
#         lower_angle = self.direction - (vision_angle / 2)
#         upper_angle = self.direction + (vision_angle / 2)
#         inter_neighbors = []
#         if (self.direction == 270 or self.direction == 90):
#             for neigh in neighbours:
#                 if (neigh.pos[1] > self.pos[1] and self.dir == 90) or (neigh.pos[1] < self.pos[1] and self.dir == 270):
#                     inter_neighbors.append(neigh)

#         # Checks if the agent is looking left uor right
#         elif (self.direction == 0 or self.direction == 180):
#             for neigh in neighbours:
#                 if (neigh.pos[0] > self.pos[0] and self.dir == (0 or 360)) or (neigh.pos[0] < self.pos[0] and self.dir == 180):
#                     inter_neighbors.append(neigh)
                    
#         else:
#             # calculate the linear formula for the line
#             m1 = math.tan(math.radians(lower_angle))
#             b1 = self.pos[1] - (m * self.pos[0])
            
#             m2 = math.tan(math.radians(upper_angle))
#             b2 = self.pos[1] - (m * self.pos[0])            

#             # calcuate the y offset of the range of lines

#             for neigh in neighbours:
#                 if ((neigh.pos[1] - ((m1 * neigh.pos[0]) + b1)) >= 0 and (neigh.pos[1] - ((m2 * neigh.pos[0]) + b2)) >= 0 and self.direction < 180):
#                     inter_neighbors.append(neigh)
#                 elif:((neigh.pos[1] - ((m1 * neigh.pos[0]) + b1)) <= 0 and (neigh.pos[1] - ((m2 * neigh.pos[0]) + b2)) <= 0 and self.direction > 180):
#                     inter_neighbors.append(neigh)
                    
#         return inter_neighbors
                    
            

    def pedestrians_in_field(self, vision_angle, vis_range, neighbours):
        """
        returns the number of pedestrians in the field
        """
        
        print(self.direction)
        # Calculate the lower angle and the upper angle
        lower_angle = self.direction - (vision_angle / 2)
        upper_angle = self.direction + (vision_angle / 2)
        
        print(lower_angle)
        print(upper_angle)

        # Change the current points to an np array for simplicity
        p0 = np.array(self.pos)

        # Convert to radians for angle calcuation
        u_rads = math.radians(upper_angle)
        l_rads = math.radians(lower_angle)
        # Calculate the end angles
        dx1 = math.cos(l_rads) * vis_range
        dy1 = math.sin(l_rads) * vis_range
        dx2 = math.cos(u_rads) * vis_range
        dy2 = math.sin(u_rads) * vis_range

        # Calculate the points
        p1 = np.array([p0[0] + dx1, p0[1] + dy1])
        p2 = np.array([p0[0] + dx2, p0[1] + dy2])
        # Calculate the vectors
        v1 = p1-p0
        v2 = p2-p0

        # Get the current neighbors
        cone_neigh = []
        # Loop to find if neighbor is within the cone
        for neigh in neighbours:
            v3 = np.array(neigh.pos) - p0
            # Append object to cone_neigh if its within vision cone
            if round((np.cross(v1, v3) * np.cross(v1, v2),5) >= 0 and round(np.cross(v2, v3) * np.cross(v2, v1),5) >= 0 and type(neigh) == Monkey):
                cone_neigh.append(neigh)

        return cone_neigh


    def update_angle(self):
        # Find the current heading
        if (self.pos != self.pre_pos):

            deltapos = self.model.space.get_heading(self.pos, self.pre_pos)
            if (deltapos[0] != 0):
                cur_angle = math.degrees(math.atan((deltapos[1] / deltapos[0])))
                self.angle = cur_angle
                print("The heading is:", cur_angle)
            else:
                if(self.dir == "up"):
                    self.angle = 90
                    print("The heading is 90")
                elif(self.dir == "down"):
                    self.angle = 270
                    print("The heading is 270")
                    
                    
    def pedestrian_intersection(self, conal_neighbours, angle, offset):
        """This fucntion will check the map for intersections from the given angle and the offset
        and return a list of neighbours that match those crieria
        Conal_neighbours: the objects within the vision field
        Angle: the direction k
        Offset: 1.5*radius_of_pedestrian to both sides of the direction line
        """

        # Checks if the agent is looking straight up or down
        neighbours = conal_neighbours
        inter_neighbors = []

        if (angle == 270 or angle == 90):
            for neigh in neighbours:
                if (neigh.pos[0] > self.pos[0] - offset and neigh.pos[0] < self.pos[0] + offset):
                    inter_neighbors.append(neigh)

        # Checks if the agent is looking left uor right
        elif (angle == 0 or angle == 180):
            for neigh in neighbours:
                if (neigh.pos[1] > self.pos[1] - offset and neigh.pos[1] < self.pos[1] + offset):
                    inter_neighbors.append(neigh)

                    # The agent is looking at an different non-exeption angle
        else:
            # calculate the linear formula for the line
            m = math.tan(math.radians(angle))
            b = self.pos[1] - (m * self.pos[0])

            # calcuate the y offset of the range of lines
            b_offset = offset / math.cos(angle)

            # calcuate the new intersection points based off the offset of the line
            b_top = b + b_offset
            b_bot = b - b_offset

            for neigh in neighbours:
                if ((neigh.pos[1] - ((m * neigh.pos[0]) + b_top)) <= 0 and (
                        neigh.pos[1] - ((m * neigh.pos[0]) + b_bot)) >= 0):
                    inter_neighbors.append(neigh)

        return inter_neighbors
    
    def closest_ped_on_line(self, neighbours, direction):
        """
        This would find the closest pedestrian to a path given a subset of pedestrians
        """
        # Find the terms for the equation for the line that will be passing through the current point in direction

        if type(neighbours) == Pedestrian:
            m = math.tan(math.radians(direction))
            b = self.pos[1] - (m*self.pos[0])
            # Calculate the first distance from the line (perpendicular distance and assign the min pedestrian
            min_distance = abs((m*neighbours.pos[0])-neighbours.pos[1]+b)/math.sqrt((m**2) + 1)
            min_pedestrian = neighbours
        else:
            m = math.tan(math.radians(direction))
            b = self.pos[1] - (m*self.pos[0])
            # Calculate the first distance from the line (perpendicular distance and assign the min pedestrian
            min_distance = abs((m*neighbours[0].pos[0])-neighbours[0].pos[1]+b)/math.sqrt((m**2) + 1)
            min_pedestrian = neighbours[0]
            # If there are more, check the rest
            if len(neighbours)>1:
                for i in range(1, len(neighbours)):
                    # calculate the distance if the current neighbour
                    cur_distance = abs((m * neighbours[i].pos[0]) - neighbours[i].pos[1] + b) / math.sqrt((m ** 2) + 1)
                    # Checks distance against that stored
                    if cur_distance < min_distance:
                        min_pedestrian = neighbours[i]
                        min_distance = cur_distance
                    # if equal checks to see which is closer to the current position.
                    elif cur_distance == min_distance:
                        if self.model.space.get_distance(self.pos, min_pedestrian.pos) > self.model.space.get_distance(self.pos, neighbours[i].pos):
                            min_pedestrian = neighbours[i]
                            min_distance = cur_distance

            # Returns the min distance and the corresponding pedestrian
        return min_distance, min_pedestrian

    def closest_pedestrian(self, inter_neigh, direction):
        """
        This is used to find the closest pedestrian of a given included list of neighbours
        Returns distance to and the position of the closest pedestrian
        TODO: check
        """
        c = self.model.space.get_distance(self.pos, inter_neigh[0].pos)
        min_neigh = inter_neigh[0]
        for i in range(1, len(inter_neigh)):
            cur_distance = self.model.space.get_distance(self.pos, inter_neigh[i].pos)
            if cur_distance < c:
                c = cur_distance
                min_neigh = inter_neigh[i]

        b = self.closest_ped_on_line(min_neigh, direction)[0]

        min_distance = math.sqrt(c**2 + b**2)

        return min_distance
    
    def theta_calc(self, ped, angle):
        """returns the angle given the pedestrians location and the directions"""
        m = math.tan(math.radians(angle))
        b = self.pos[1] - (m * self.pos[0])

        if ped.pos[1] - (m*ped.pos[0])+b < 0 and ped.direction>angle and ped.direction < angle+math.radians(180):
            return abs(angle - ped.direction)
        elif ped.pos[1] - (m*ped.pos[0])+b > 0 and (ped.direction<angle or ped.direction > angle+math.radians(180)):
            return abs(angle - ped.direction)
        else:
            return 0
        
    def check_theta(self, peds_in_180, direction):
        if len(peds_in_180)>0:
            cpil = self.closest_ped_on_line(peds_in_180, direction)[1]
            theta_vj = self.theta_calc(cpil,direction)
        else:
            theta_vj = 0
        return theta_vj

In [85]:
def test_field(pos_1, angle_1, pos_2, angle_2, pos_3, angle_3, vision_range, angle):
    tester = TestModel(10, 10)

    # Create a Monkey
    tester.new_agent(Monkey, pos_1, angle_1)
    tester.new_agent(Monkey, pos_2, angle_2)
    tester.new_agent(Monkey, pos_3, angle_3)

    # Create a reference, so that we can properly test
    Bernard = tester.agents[0]
    print("Bernard", Bernard.pos)

    Babette = tester.agents[1]
    print('Babette', Babette.pos)
    
    Bungo = tester.agents[2]
    print('Bungo', Bungo.pos)
    
    neighbours = Bernard.model.space.get_neighbors(Bernard.pos, include_center=False, radius=3)
    #num_seen = Bernard.pedestrians_in_field(angle,2, neighbours)
    num_seen = Bernard.pedestrian_intersection(neighbours, Bernard.direction, .5)
    print(len(num_seen))
    return len(num_seen)

In [93]:
# Check the ped line functions 
# VISION FIELD SET AT 180 DEGREES
# ASSUMING THE CORRECT MAP
# VISION RANGE = 1

angle = 180

assert test_field((1,1),180, (0,1.25),90,(.5,0),180, 1, angle) == 1, 'Well somethings wrong'

# results = test_field2((0,0), (.1,.1),(.2,.2),(.3,.3), 1, angle)
# assert result == 1, 'We arent seeing what I think we should'
# print()

Bernard (1, 1)
Babette (0, 1.25)
Bungo (0.5, 0)
1


In [9]:
def test_field(pos_1, pos_2, vision_range):
    tester = TestModel(10, 10)

    # Create a Monkey
    tester.new_agent(Monkey, pos_1)

    # Create a reference, so that we can properly test
    Bernard = tester.agents[0]
    print("Bernard", Bernard.pos)
    Bernard.update_angle()

    Bernard.move(pos_2)
    Bernard.update_angle()
    return Bernard.pedestrians_in_field(180, vision_range)



In [10]:
# VISION FIELD SET AT 180 DEGREES
# ASSUMING THE CORRECT MAP
# VISION RANGE = 1

# Check if for bernard at 0,0, he sees babette at 1/2*sqrt(2),1/2*sqrt(2), which corresponds to radius of 1/4pi
# assert len(test_field((0,0), ((2)**.5/2-.001, (2)**.5/2-.001), 1))>0, 'Bernard cannot see Babette! 1 :(('
test_field((0,0), ((2)**.5/2-.001, (2)**.5/2-.001), 1)

Bernard (0, 0)
The heading is: 45.0
